In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern, ConstantKernel, PairwiseKernel
import matplotlib.pyplot as plt
import matplotlib
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
from tqdm import tqdm
import time
np.random.seed(24)
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


## Data Preprocessing

In [2]:
data = np.loadtxt('data.txt', dtype='float64',delimiter=',')
MAX = np.max(data, axis=0)
MIN = np.min(data, axis=0)
norm = ((data - MIN) / (MAX - MIN) + 0.1) * 4 - 2.4

In [3]:
np.random.shuffle(norm)

In [5]:
X = norm[:,:-1]
Y = norm[:,-1:]
num_train = 80
num_test = 20
train_X = X[:num_train,:]
train_Y = Y[:num_train,:]
test_X = X[-num_test:,:]
test_Y = Y[-num_test:,:]

## Gaussian Process

In [6]:
## initialize and fit model matern
# kernel = DotProduct() + WhiteKernel()
t1 = time.time()
Matern_kernel = Matern(length_scale=2, nu=3/2)
Poly_kernel = ConstantKernel() * PairwiseKernel(metric='poly', pairwise_kernels_kwargs=dict(degree=4))
gpr = GaussianProcessRegressor(kernel=Poly_kernel,random_state=0).fit(train_X, train_Y)
t2 = time.time()
print(t2-t1)

0.05438733100891113


In [7]:
## test model
t1 = time.time()
gpr_pred_Y, gpr_std_Y = gpr.predict(test_X, return_std=True)
t2 = time.time()
print(t2-t1)

0.007851839065551758


/home/yinanw/anaconda3/envs/tensorflow3.6/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:346: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [ ]:
gpr_std_Y

In [ ]:
## denormalize
test_Y_denorm = (test_Y-0.1) * (MAX - MIN)[-1] + MIN[-1]
gpr_pred_Y_denorm = (gpr_pred_Y-0.1) * (MAX - MIN)[-1] + MIN[-1]

## calculate the mean absolute percentage error of prediction
gpr_MAPE = np.mean(np.abs((gpr_pred_Y - test_Y)/test_Y))
gpr_RMSE = np.sqrt(np.mean(np.square(test_Y - gpr_pred_Y)))
print("The Mean Absolute Percentage Error of Gaussian Process is: ",gpr_MAPE)
print("The Rooted Mean Squared Error of Gaussian Process is: ",gpr_RMSE)

In [ ]:
pred_y = gpr_pred_Y
target_y = Y
context_y = train_Y

In [ ]:
filename = 'UQ/results/30_GP_Poly.npz'
np.savez(filename, pred = pred_y, target = target_y[-num_test:], std = gpr_std_Y)

In [ ]:
gpr_std_Y.shape

In [ ]:
labelfont = 30
markerline = 25
markerstar=17

matplotlib.rc('xtick', labelsize=labelfont) 
matplotlib.rc('ytick', labelsize=labelfont) 
plt.figure(figsize=(28,20))
plt.plot(pred_y, 'r_', markersize=markerline, mew=4, label='Prediction')
plt.plot(target_y[-num_test:], 'b*', markersize=markerstar, label='Ground Truth')
plt.errorbar(np.arange(num_test), pred_y, gpr_std_Y*1.96, linestyle='None', capsize=12,
             capthick=5, elinewidth=5, ecolor='c', label='Confidence Interval')
plt.grid('off')
plt.legend(fontsize=labelfont, loc='upper right')
ax = plt.gca()
plt.xlim(-1,21)
plt.ylim(-2.5,2.5)
plt.xlabel('index of data points',fontsize=labelfont)
# plt.ylabel('value of y',fontsize=labelfont)
#plt.savefig('test_80_Poly_range4.png')
# plt.show()